In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime as d

import lxml
import requests as r
import json

In [ ]:
#get links of all categories
def getCatLinks():
    main_url = 'http://edition.cnn.com/'
    res = r.get(main_url)
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    tags = soup.select('.m-footer__bucket_group .m-footer__link')
    
    catList = []
    for tag in tags:
        if (tag['href'].split('.')[0] == '//money'):
            catList.append('http:%s'%(tag['href']))
        elif (tag['href'].split(':')[0] != 'http'):
            catList.append(main_url + '%s'%(tag['href'].lstrip('/')))
    return catList

In [ ]:
#[DEBUG]
for link in getCatLinks():
    res = r.get(link)
    if res.status_code == 200:
        print('[ok!]' + link)
    else:
        print('[%s Fail!]'%(res.status_code) + link)

In [ ]:
#get all links of articles in each category
def getArticlesList(cat_url):
    
    articlesList = []

    res = r.get(cat_url)
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    titles = soup.select('.cd__headline a')
    
    for article in titles:
        art_info = {}
        art_info['title'] = article.text 
        
        if (article['href'].lstrip('/').split(':')[0] == 'https'):
            art_info['url'] = article['href'].lstrip('/')
        elif (article['href'].lstrip('/').split(':')[0] == 'http'):
            art_info['url'] = article['href'].lstrip('/')
        else:
            art_info['url'] = 'http://edition.cnn.com/' + article['href'].lstrip('/')
        
        articlesList.append(art_info)
        
    return articlesList

In [ ]:
#[DEBUG]
for catLink in getCatLinks()[0:1]:
    print(len(getArticlesList(catLink)))
    
    for art in getArticlesList(catLink):
        res = r.get(art['url'])
        if res.status_code == 200:
            print('[ok!]' + art['title'])
        else:
            print('[%s Fail!]'%(res.status_code) + art['url'])    

In [ ]:
getArticlesList(getCatLinks()[0])[10]['url']

In [ ]:
#get a signle article content
def getArticle(art_url):
    
    art_dict = {}

    res = r.get(art_url)
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    art_date = soup.find(itemprop = 'datePublished')['content'].split('T')[0]
    art_title = soup.select('.pg-headline')[0].text
    art_body = soup.select('#body-text .zn-body__paragraph')

    art_paras = {} 
    for numPara in range(len(art_body)):
        art_paras[numPara+1] = art_body[numPara].text.lstrip('•')

    art_dict['title'] = art_title
    art_dict['url'] = art_url
    art_dict['source'] = 'CNN'
    art_dict['date'] = art_date
    art_dict['content'] = art_paras
            
    return art_dict

In [ ]:
getArticle(getArticlesList(getCatLinks()[0])[10]['url'])

In [ ]:
#先不要用
def addContent(articlesList):
    
    arts_dictList = []
    
    for num in range(len(articlesList)):
        art_info = {}
        
        res = r.get(articlesList[num]['url'])
        res.encoding='unicode'
        soup = BeautifulSoup(res.text,'lxml')       
        art_body = soup.select('#body-text .zn-body__paragraph')
        date = soup.select('.pg-rail-tall meta')[1]['content'].split('T')[0]
        
        para_dict = {} 
        for numPara in range(len(art_body)):
            para_dict[numPara+1] = art_body[numPara].text.lstrip('•')
        
        art_info['title'] = articlesList[num]['title']
        art_info['date'] = date
        art_info['source'] = 'CNN'
        art_info['url'] = articlesList[num]['url']
        art_info['content'] = para_dict
        
        arts_dictList.append(art_info)
            
    return arts_dictList

In [ ]:
res = r.get('http://edition.cnn.com/2017/04/15/us/atlanta-sun-dial-restaurant-boy-dies/index.html')
res.encoding='unicode'
soup = BeautifulSoup(res.text,'lxml')       
date = soup.select('.pg-rail-tall meta')[1]['content'].split('T')[0]

print(date)

In [ ]:
addContent(getArticlesList(getCatLinks()[0])[0:2])

In [ ]:
def jsonOut(inputList):
    src = inputList[0]['source']
    day = d.today().strftime('%Y%m%d')
    filename = '{}_{}_output.json'.format(day,src)
    with open(filename, 'w') as f:
        json.dump(inputList, f)
        f.close()
    print(filename + ' has been created!')

In [ ]:
jsonOut(addContent(test))